# Projeto

## Data Preparation

- Eliminar Colunas IDentificadoras e tornar ID como PK

- Eliminar Erros de Input
    - Format "Age" & Check SSN pattern
    - Negative Numeric Variables: 'Age', 'Num_Bank_Accounts', 'NumofLoan'
    - Age Restriction (GREATER than 0 && LESS than 150)
<br><br>
- Eliminar 9 Records de 'Monthly_Balance' igual a -3.33333E+26

- Trabalhar com 'Credit_History_Age' como variavel numerica 

- Variable Enconding of 'Age' and 'Credit_History_Age'

In [1]:
import pandas as pd
import re
import numpy as np
import dslabs_functions as dslab
from math import pi, sin, cos

In [2]:
file_tag = 'credit_score'
filename = 'class_credit_score.csv'
original_dataset: pd.DataFrame = pd.read_csv(filename, index_col='ID', sep=',', decimal='.', na_values='')

# Drop IDentifying Columns
dataset_cleaned = original_dataset.drop(columns=['Customer_ID', 'Name', 'SSN'], errors='ignore')

# Erase Input Errors
## Check and Clean 'SSN' & 'Age' Column
pattern = r'^\d{3}-\d{2}-\d{4}$'  # Replace this with your SSN regex pattern
#dataset_cleaned['SSN'] = dataset_cleaned['SSN'].apply(lambda x: x if re.match(pattern, str(x)) else np.nan)
dataset_cleaned['Age'] = dataset_cleaned['Age'].astype(str).str.replace('_', '', regex=False)

## Transfrom 'Age' from SIMBOLIC to NUMERIC
dataset_cleaned['Age'] = pd.to_numeric(dataset_cleaned['Age'], errors='coerce')

## Invalid Numeric Variables 'Age', 'Num_Bank_Accounts', 'NumofLoan' Column
dataset_cleaned['Age'] = dataset_cleaned['Age'].apply(lambda x: np.nan if x == -500 or x >= 150 else x)
dataset_cleaned['Num_Bank_Accounts'] = dataset_cleaned['Num_Bank_Accounts'].apply(lambda x: x if x >= 0 else np.nan)
dataset_cleaned['NumofLoan'] = dataset_cleaned['NumofLoan'].apply(lambda x: x if x >= 0 else np.nan)

# Monthly_Balance 
dataset_cleaned['MonthlyBalance'] = dataset_cleaned['MonthlyBalance'].apply(lambda x: x if x >= 0 else np.nan)

## Transform 'Credit_History_Age' from SIMBOLIC to NUMERIC
hist_age = dataset_cleaned["Credit_History_Age"]
hist_vals = []

for val in hist_age:
    matches = re.findall(r'(\d+) Years and (\d+) Months', str(val))
    if not matches:  # Use `not matches` instead of `matches == []`
        hist_vals.append(np.nan)
    else:
        hist_vals.append(float(matches[0][0]) + (float(matches[0][1]) / 12))

dataset_cleaned["Credit_History_Age"] = hist_vals

### Drop 'Credit_History_Age' as OBJECT 
object_columns = dataset_cleaned.select_dtypes(include=['object']).columns.tolist()
if "CreditHistoryAge" in object_columns:
    dataset_cleaned.drop(columns=["CreditHistoryAge"], inplace=True)

# Get Variables Type Right (Object to Category)
variables_types: dict[str, list] = dslab.get_variable_types(dataset_cleaned)
numeric: list[str] = variables_types["numeric"]
symbolic: list[str] = variables_types["symbolic"]
binary: list[str] = variables_types["binary"]

print(variables_types)

# Object to Category (same as symbolic in Pandas)
dataset_cleaned[symbolic] = dataset_cleaned[symbolic].apply(lambda x: x.astype("category"))
print(dataset_cleaned.dtypes)

{'numeric': ['Age', 'Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts', 'Num_Credit_Card', 'Interest_Rate', 'NumofLoan', 'Delay_from_due_date', 'NumofDelayedPayment', 'ChangedCreditLimit', 'NumCreditInquiries', 'OutstandingDebt', 'CreditUtilizationRatio', 'Credit_History_Age', 'TotalEMIpermonth', 'Amountinvestedmonthly', 'MonthlyBalance'], 'binary': ['Credit_Score'], 'date': [], 'symbolic': ['Month', 'Occupation', 'Type_of_Loan', 'CreditMix', 'Payment_of_Min_Amount', 'Payment_Behaviour']}
Month                     category
Age                        float64
Occupation                category
Annual_Income              float64
Monthly_Inhand_Salary      float64
Num_Bank_Accounts          float64
Num_Credit_Card              int64
Interest_Rate                int64
NumofLoan                  float64
Type_of_Loan              category
Delay_from_due_date          int64
NumofDelayedPayment        float64
ChangedCreditLimit         float64
NumCreditInquiries         float64
Credi

### Variable Enconding

- *'Credit_History_Age' feito na 1ª célula*

1. Order Enconding Binary *(order is irrelevant)*
    - 'Credit_Score' <br><br>

2. Order Enconding *(order is relevant)*
    - 'CreditMix' [Bad, Standard, Good] (3)<br><br>
    - 'Payment_of_Min_Amount' ['No', 'NM', 'Yes'] (3) <br><br>
    - 'Payment_Behaviour' ['Low_spent_Small_value_payments', 'Low_spent_Medium_value_payments', 'Low_spent_Large_value_payments', 
        'High_spent_Small_value_payments', 'High_spent_Medium_value_payments', 'High_spent_Large_value_payments'] (6) <br><br>

3. Cyclic Variables
    - 'Month' ordenar primeiro [January, February, March, ...]<br><br>

4. Dummification/ One-Hot Enconding
    - 'Type_of_Loan' ['Student Loan', 'Payday Loan', 'Not Specified', 'Home Equity Loan', 'Auto Loan', 
        'Personal Loan', 'Debt Consolidation Loan', 'Credit-Builder Loan', 'Mortgage Loan']<br><br>
    - 'Occupation' 

In [3]:
# Collect Individual values for each symbolic var, in order to encode the rest
dataset_cleaned

for v in variables_types["symbolic"]:
    print(v, dataset_cleaned[v].unique())

Month ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August']
Categories (8, object): ['April', 'August', 'February', 'January', 'July', 'June', 'March', 'May']
Occupation ['Scientist', NaN, 'Teacher', 'Engineer', 'Entrepreneur', ..., 'Accountant', 'Musician', 'Mechanic', 'Writer', 'Architect']
Length: 16
Categories (15, object): ['Accountant', 'Architect', 'Developer', 'Doctor', ..., 'Musician', 'Scientist', 'Teacher', 'Writer']
Type_of_Loan ['Auto Loan, Credit-Builder Loan, Personal Loan..., 'Credit-Builder Loan', 'Auto Loan, Auto Loan, and Not Specified', 'Not Specified', NaN, ..., 'Auto Loan, Payday Loan, Auto Loan, Student Lo..., 'Home Equity Loan, Payday Loan, Not Specified,..., 'Home Equity Loan, Auto Loan, Auto Loan, and A..., 'Payday Loan, Student Loan, Mortgage Loan, and..., 'Personal Loan, Auto Loan, Mortgage Loan, Stud...]
Length: 6261
Categories (6260, object): ['Auto Loan', 'Auto Loan, Auto Loan, Auto Loan, Auto Loan, C..., 'Auto Loan, Auto Loan, Auto L

In [4]:
# Enconding 'Data_Score' Binary - Ordinal Enconding (easiest one)
variables_types

yes_no: dict[str, int] = {"no": 0, "No": 0, "yes": 1, "Yes": 1}
credit_score_values: dict[str, int] = {"Poor": 0, "Good": 1}

encoding_binary: dict[str, dict[str, int]] = {
    "Credit_Score": credit_score_values,
}

data_encoded_binary: pd.DataFrame = dataset_cleaned.replace(encoding_binary, inplace=False)

# Enconding 'Credit_Mix' Non-Binary - Ordinal Enconding
credit_mix_values: dict[str, int] = {"Bad": 0, "Standard": 1, "Good": 2}

# Enconding 'Payment_of_Min_Amount' Non-Binary - Ordinal Enconding
payment_of_min_amount_values: dict[str, int] = {"No": 0, "NM": 1, "Yes": 2}

# Enconding 'Payment_Behaviour' Non-Binary - Ordinal Enconding
payment_behaviour_values: dict[str, int] = {"Low_spent_Small_value_payments": 0, "Low_spent_Medium_value_payments": 1, "Low_spent_Large_value_payments": 2,
                                             "High_spent_Small_value_payments": 3, "High_spent_Medium_value_payments": 4, "High_spent_Large_value_payments": 5}

encoding_non_binary: dict[str, dict[str, int]] = {
    "CreditMix": credit_mix_values,
    "Payment_of_Min_Amount": payment_of_min_amount_values,
    "Payment_Behaviour": payment_behaviour_values,
}

data_encoded_non_binary: pd.DataFrame = data_encoded_binary.replace(encoding_non_binary, inplace=False)

# Enconding 'Month' - Cyclic variables
month_values: dict[str, float] = {
    "January": 0,
    "February": 1/6 * pi,
    "March": 2/6 * pi,
    "April": 3/6 * pi,
    "May": 4/6 * pi,
    "June": 5/6 * pi,
    "July": pi,
    "August": -5/6 * pi,
    "September": -4/6 * pi,
    "October": -3/6 * pi,
    "November": -2/6 * pi,
    "December": -1/6 * pi,
}

enconding_cyclic: dict[str, dict] = {
    "Month": month_values,
}

data_encoded_cyclic = data_encoded_non_binary.replace(enconding_cyclic)
data_encoded_cyclic.head(3)

## QUAL E A IDEIA DISTO???? (se manter nao mudar nome variavel)
# data_encoded_cyclic: pd.DataFrame | None = dslab.encode_cyclic_variables_2(data_encoded_cyclic, ["Month"])
# if data_encoded_cyclic is not None:
#     data_encoded_cyclic.head()

,Month,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,NumofLoan,Type_of_Loan,...,CreditMix,OutstandingDebt,CreditUtilizationRatio,Credit_History_Age,Payment_of_Min_Amount,TotalEMIpermonth,Amountinvestedmonthly,Payment_Behaviour,MonthlyBalance,Credit_Score
ID,,,,,,,,,,,,,,,,,,,,,
0x1602,0.000000,23.0,Scientist,19114.12,1824.843333,3.0,4,3,4.0,"Auto Loan, Credit-Builder Loan, Personal Loan,...",...,NaN,809.98,26.822620,22.083333,0,49.574949,80.415295,3,312.494089,1
0x1603,0.523599,23.0,Scientist,19114.12,NaN,3.0,4,3,4.0,"Auto Loan, Credit-Builder Loan, Personal Loan,...",...,2,809.98,31.944960,NaN,0,49.574949,118.280222,2,284.629163,1
0x1604,1.047198,NaN,Scientist,19114.12,NaN,3.0,4,3,4.0,"Auto Loan, Credit-Builder Loan, Personal Loan,...",...,2,809.98,28.609352,22.250000,0,49.574949,81.699521,1,331.209863,1


In [5]:
# Enconding 'Type_of_Loan' - Dummification
type_of_loans = data_encoded_cyclic["Type_of_Loan"]
unique_loans_list = []
credit_score_index = data_encoded_cyclic.columns.get_loc("Credit_Score")

for loan in type_of_loans:
    
    matches = re.findall(r'([\w\s-]+)(?:, | and |$)', str(loan))
    m_filtered = [m if "and" not in m else m[4:] for m in matches]
    m_filtered_series = pd.Series(m_filtered)

    unique_loans_list.extend(m_filtered)

# drop nan in list of unique values (don't use it for enconding)
unique_loans_list = list(set(unique_loans_list))
unique_loans_list = [item for item in unique_loans_list if item != 'nan']
#['Student Loan', 'Payday Loan', 'Not Specified', 'Home Equity Loan', 'Auto Loan', 
# 'Personal Loan', 'Debt Consolidation Loan', 'Credit-Builder Loan', 'Mortgage Loan']

# Create new Columns for Dummification
for loan_type in unique_loans_list:
    data_encoded_cyclic.insert(credit_score_index, loan_type, 0)

# Check which Loans there is in the original column and update Loan types
for index, row in data_encoded_cyclic.iterrows():
    types = re.findall(r'([\w\s-]+)(?:, | and |$)', str(row["Type_of_Loan"]))
    types_filtered = [t if "and" not in t else t[4:] for t in types]
    
    for loan_type in types_filtered:
        if loan_type in unique_loans_list:
            data_encoded_cyclic.at[index, loan_type] = 1

# Drop original "Type_of_Loan"
data_encoded_cyclic.drop("Type_of_Loan", axis=1, inplace=True)

data_encoded_cyclic.head(9)

,Month,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,NumofLoan,Delay_from_due_date,...,Personal Loan,Student Loan,Home Equity Loan,Debt Consolidation Loan,Mortgage Loan,Auto Loan,Payday Loan,Credit-Builder Loan,Not Specified,Credit_Score
ID,,,,,,,,,,,,,,,,,,,,,
0x1602,0.000000,23.0,Scientist,19114.12,1824.843333,3.0,4,3,4.0,3,...,1,0,1,0,0,1,0,1,0,1
0x1603,0.523599,23.0,Scientist,19114.12,NaN,3.0,4,3,4.0,-1,...,1,0,1,0,0,1,0,1,0,1
0x1604,1.047198,NaN,Scientist,19114.12,NaN,3.0,4,3,4.0,3,...,1,0,1,0,0,1,0,1,0,1
0x1605,1.570796,23.0,Scientist,19114.12,NaN,3.0,4,3,4.0,5,...,1,0,1,0,0,1,0,1,0,1
0x1606,2.094395,23.0,Scientist,19114.12,1824.843333,3.0,4,3,4.0,6,...,1,0,1,0,0,1,0,1,0,1
0x1607,2.617994,23.0,Scientist,19114.12,NaN,3.0,4,3,4.0,8,...,1,0,1,0,0,1,0,1,0,1
0x1608,3.141593,23.0,Scientist,19114.12,1824.843333,3.0,4,3,4.0,3,...,1,0,1,0,0,1,0,1,0,1
0x1609,-2.617994,23.0,Scientist,19114.12,1824.843333,3.0,4,3,4.0,3,...,1,0,1,0,0,1,0,1,0,1
0x160e,0.000000,28.0,NaN,34847.84,3037.986667,2.0,4,6,1.0,3,...,0,0,0,0,0,0,0,1,0,1
